In [2]:

import requests
from bs4 import BeautifulSoup
import time
from urllib.parse import urljoin, urlparse

# クロール対象のトップページ
base_url = "https://www.musashino-u.ac.jp/"

# 訪問済みURLと結果を格納する辞書
visited = set()
url_title_dict = {}

# リクエスト設定
delay = 0.25   # 秒
max_pages = 3000  # 安全上限

# 除外対象の拡張子
exclude_ext = (
    ".pdf", ".png", ".jpg", ".jpeg", ".gif",
    ".svg", ".zip", ".mp4", ".mov", ".pptx", ".docx", ".xlsx"
)

def is_valid_link(href):
    """リンクが有効で、クロール対象か判定"""
    if not href or href.startswith("#") or "javascript:" in href:
        return False
    if any(href.lower().endswith(ext) for ext in exclude_ext):
        return False
    return True

def crawl(url):
    """再帰的クロール"""
    if url in visited or len(visited) >= max_pages:
        return
    visited.add(url)

    print(f"📘 Accessing: {url}")
    try:
        time.sleep(delay)
        response = requests.get(url, timeout=10)
        response.raise_for_status()
    except Exception as e:
        print(f"⚠️ Error accessing {url}: {e}")
        return

    soup = BeautifulSoup(response.text, "html.parser")

    # <title> 抽出
    title_tag = soup.find("title")
    title_text = title_tag.get_text().strip() if title_tag else "No Title"
    url_title_dict[url] = title_text

    # 内部リンクを探索
    for a_tag in soup.find_all("a", href=True):
        href = a_tag["href"].strip()
        if not is_valid_link(href):
            continue
        full_url = urljoin(url, href)
        parsed = urlparse(full_url)
        # 同一ドメイン内のみ
        if parsed.netloc == urlparse(base_url).netloc:
            crawl(full_url)

# 実行
crawl(base_url)

# 結果出力
print("\n=== 収集結果 ===")
for url, title in url_title_dict.items():
    print(f"{url}: {title}")

print(f"\n合計 {len(url_title_dict)} ページを取得しました。")


📘 Accessing: https://www.musashino-u.ac.jp/
📘 Accessing: https://www.musashino-u.ac.jp/access.html
📘 Accessing: https://www.musashino-u.ac.jp/admission/request.html
📘 Accessing: https://www.musashino-u.ac.jp/contact.html
📘 Accessing: https://www.musashino-u.ac.jp/prospective-students.html
📘 Accessing: https://www.musashino-u.ac.jp/students.html
📘 Accessing: https://www.musashino-u.ac.jp/alumni.html
📘 Accessing: https://www.musashino-u.ac.jp/parents.html
📘 Accessing: https://www.musashino-u.ac.jp/business.html
📘 Accessing: https://www.musashino-u.ac.jp/guide/
📘 Accessing: https://www.musashino-u.ac.jp/guide/profile/
📘 Accessing: https://www.musashino-u.ac.jp/guide/activities/
📘 Accessing: https://www.musashino-u.ac.jp/guide/campus/
📘 Accessing: https://www.musashino-u.ac.jp/guide/facility/
📘 Accessing: https://www.musashino-u.ac.jp/guide/information/
📘 Accessing: https://www.musashino-u.ac.jp/guide/profile/media/
📘 Accessing: https://www.musashino-u.ac.jp/admission/
📘 Accessing: https:/